In [6]:
import nibabel as nib
import numpy as np
from scipy.spatial.distance import directed_hausdorff
from scipy.ndimage import binary_erosion

def evaluate_3d_metrics(pred_path, gt_path, num_classes):
    # Load the predicted and ground truth NIfTI files
    pred_nii = nib.load(pred_path)
    gt_nii = nib.load(gt_path)
    
    # Get the data arrays from the NIfTI objects
    pred_volume = pred_nii.get_fdata()
    gt_volume = gt_nii.get_fdata()
    
    print("Unique values in prediction volume:", np.unique(pred_volume))
    print("Unique values in ground truth volume:", np.unique(gt_volume))
    
    # Create a mapping dictionary for the predicted labels
    mapping = {0: 0, 63: 1, 126: 2, 189: 3, 252: 4}

    # Apply the mapping to the predicted volume
    remapped_pred_volume = np.copy(pred_volume)
    for original_value, new_value in mapping.items():
        remapped_pred_volume[pred_volume == original_value] = new_value
        
    pred_volume = remapped_pred_volume
    
    print("Unique values in prediction volume:", np.unique(pred_volume))
    print("Unique values in ground truth volume:", np.unique(gt_volume))
    
    
    print(pred_volume.shape)

    # Check shape
    assert pred_volume.shape == gt_volume.shape, "Shape mismatch between prediction and ground truth volumes."

    metrics = {}
    # Skip background (class 0)
    for c in range(1, num_classes):  
        # print(c)
        pred_mask = (pred_volume == c).astype(np.uint8)
        gt_mask = (gt_volume == c).astype(np.uint8)
        
        
        # Compute 3D IoU
        intersection = np.sum(pred_mask & gt_mask)
        union = np.sum(pred_mask | gt_mask)
        iou = intersection / union if union != 0 else 0
        metrics[f"IoU_Class_{c}"] = iou

        print("IOU", iou)

        # Compute 3D Dice Score
        dice = (2 * intersection) / (np.sum(pred_mask) + np.sum(gt_mask)) if (np.sum(pred_mask) + np.sum(gt_mask)) != 0 else 0
        metrics[f"Dice_Class_{c}"] = dice

        print("Dice", dice)

        # Compute 3D Hausdorff Distance
        pred_boundary = pred_mask - binary_erosion(pred_mask)
        gt_boundary = gt_mask - binary_erosion(gt_mask)
        pred_boundary_points = np.argwhere(pred_boundary)
        gt_boundary_points = np.argwhere(gt_boundary)
        
        if pred_boundary_points.size > 0 and gt_boundary_points.size > 0:
            hd_pred_to_gt = directed_hausdorff(pred_boundary_points, gt_boundary_points)[0]
            hd_gt_to_pred = directed_hausdorff(gt_boundary_points, pred_boundary_points)[0]
            hausdorff_distance = max(hd_pred_to_gt, hd_gt_to_pred)
        else:
            hausdorff_distance = np.inf

        metrics[f"Hausdorff_Class_{c}"] = hausdorff_distance
        print("Hausdorff", hausdorff_distance)

        if c != 0:
            # Compute 3D ASSD (Average Symmetric Surface Distance)
            if pred_boundary_points.size > 0 and gt_boundary_points.size > 0:
                assd_pred_to_gt = np.mean([np.min(np.linalg.norm(pred_boundary_points - point, axis=1)) for point in gt_boundary_points])
                assd_gt_to_pred = np.mean([np.min(np.linalg.norm(gt_boundary_points - point, axis=1)) for point in pred_boundary_points])
                assd = (assd_pred_to_gt + assd_gt_to_pred) / 2
            else:
                assd = np.inf
            
            metrics[f"ASSD_Class_{c}"] = assd
            print("ASSD", assd)

        # Compute 3D Volumetric Similarity
        gt_vol = np.sum(gt_mask, dtype=np.float64)
        pred_vol = np.sum(pred_mask, dtype=np.float64)

        vol_sim = 1 - abs(gt_vol - pred_vol) / (gt_vol + pred_vol) if (gt_vol + pred_vol) != 0 else 0
        metrics[f"VolSim_Class_{c}"] = vol_sim
        print("VOL Sim", vol_sim)

    return metrics

In [ ]:
import os
import glob
import pandas as pd

def evaluate_all_metrics(pred_folder, gt_folder, num_classes):
    # Find all prediction files in the prediction folder
    pred_files = glob.glob(os.path.join(pred_folder, '*.nii.gz'))
    
    # List to store metrics for each patient
    metrics_list = []
    
    for pred_file in pred_files:
        # Extract the patient ID from the prediction file name
        patient_id = os.path.basename(pred_file).split('.')[0]
        
        # Construct the corresponding ground truth file path
        gt_file = os.path.join(gt_folder, patient_id, 'GT_corrected.nii.gz')
        
        # Check if the ground truth file exists
        if os.path.exists(gt_file):
            # Evaluate metrics for this patient
            metrics = evaluate_3d_metrics(pred_file, gt_file, num_classes)
            # Add the patient ID to the metrics
            metrics['Patient_ID'] = patient_id
            # Append metrics to the list
            metrics_list.append(metrics)
        else:
            print(f"Ground truth file not found for {patient_id}. Skipping.")
    
    # Convert the metrics list to a DataFrame
    metrics_df = pd.DataFrame(metrics_list)
    return metrics_df

# Example usage
pred_folder = 'volumes/segthor/results_AdamW_K=25_best_epoch/'
gt_folder = 'data/segthor_train/train/'
metrics_df = evaluate_all_metrics(pred_folder, gt_folder, num_classes=5)

# Save the DataFrame to a CSV file
metrics_df.to_csv('metrics_results_results_AdamW_K=25_best_epoch.csv', index=False)

# Calculate averages, min, and max for each metric across all patients
summary_stats = metrics_df.describe().loc[['mean', 'min', 'max']]

print("Summary Statistics:")
print(summary_stats)

# Save summary statistics to a CSV file
summary_stats.to_csv('metrics_summary_results_AdamW_K=25_best_epoch.csv')


Unique values in prediction volume: [  0.  63. 126. 189. 252.]
Unique values in ground truth volume: [0. 1. 2. 3. 4.]
